# Analysis Template

A template with the tools to get started pulling data from a local db.

In [26]:
import os
import asyncio
import requests
import pandas as pd
import json
from gql import Client, gql
from gql.transport.aiohttp import AIOHTTPTransport
from decimal import Decimal
import sqlite3


In [27]:
## constants
INFURA_KEY = os.getenv('INFURA_KEY')

# set up sqlite connection
con = sqlite3.connect("../../data/perps.db")


In [28]:
df_trades = pd.read_sql_query('SELECT * FROM trades WHERE accountType = "smart_margin" ORDER BY timestamp desc', con)
df_trades.head(5)

,id,account,abstractAccount,accountType,timestamp,asset,marketKey,orderType,price,margin,positionId,feesPaid,pnl,size,positionSize,positionClosed
0,0x048c934f1a55523a70226c182742e5a7d19eea70c08e...,0xafa306dbb13beb550511ffbd304d97461e71603c,0x91bf6505b1126bb3cf430ff76466a30b0afca75d,smart_margin,1681906430,sETH,sETHPERP,DelayedOffchain,1977.099674,14671.786434,0x2b3bb4c683bfc5239b029131eef3b1d214478d93-0x382b,193.061080,-1458.900130,-193.2741,0.0000,1
1,0x06e1ecd1dea8185f97b0df173c22eefa9befcbb03546...,0xafa306dbb13beb550511ffbd304d97461e71603c,0x91bf6505b1126bb3cf430ff76466a30b0afca75d,smart_margin,1681904583,sETH,sETHPERP,DelayedOffchain,1984.024494,16238.056643,0x2b3bb4c683bfc5239b029131eef3b1d214478d93-0x382b,220.242694,0.000000,110.0000,193.2741,0
2,0xb6cb754a5174d297ec15324a0bf69b7c49e28b3e9c3a...,0xafa306dbb13beb550511ffbd304d97461e71603c,0x91bf6505b1126bb3cf430ff76466a30b0afca75d,smart_margin,1681904388,sETH,sETHPERP,DelayedOffchain,1985.471664,16526.250000,0x2b3bb4c683bfc5239b029131eef3b1d214478d93-0x382b,167.338366,0.000000,83.2741,83.2741,0
3,0x96e1dd3a9c7e93aa19bc3eea99fc5567df8c70cee7ed...,0xb7babe35ce543e2cf2f615cb1c792a2025feb572,0xcff96c386b403540a4fe40be7ee5b830fcd03bc3,smart_margin,1681893144,sETH,sETHPERP,DelayedOffchain,1986.658708,2217.785590,0x2b3bb4c683bfc5239b029131eef3b1d214478d93-0x34d6,12.120834,-599.559101,-5.0944,0.0000,1
4,0x01212aced82ad0eff683bc15e6311de7434361b9723f...,0x0f8c87b97596f2b869c794f8bef21ba69a2aef0a,0x485ed05a07fdfe6466912d91a5d21134a27ea1af,smart_margin,1681863022,SOL,sSOLPERP,DelayedOffchain,24.829264,45.994042,0x0ea09d97b4084d859328ec4bf8ebcf9ecca26f1d-0x502,3.501412,-0.035809,50.3912,0.0000,1


In [29]:
df_orders = pd.read_sql_query('SELECT * FROM orders ORDER BY timestamp desc', con)
df_orders.head(15)

,id,size,marketKey,account,txnHash,abstractAccount,orderId,targetPrice,marginDelta,timestamp,orderType,status,keeper
0,D-GBP-0x783f1477e908e5906d0f2f77ee5ca859bd83ea...,-2.581547e+04,0x73474250504552500000000000000000000000000000...,0x783f1477e908e5906d0f2f77ee5ca859bd83ea83,0x0000000000000000000000000000000000000000,0x783f1477e908e5906d0f2f77ee5ca859bd83ea83,0.0,0,0,1681938781,DelayedOffchain,Pending,0x0000000000000000000000000000000000000000
1,D-MATIC-0x77e65c9bf35bfc436c818062fbc1ff5fe09f...,1.000000e-18,0x734d4154494350455250000000000000000000000000...,0x77e65c9bf35bfc436c818062fbc1ff5fe09fb11c,0x0000000000000000000000000000000000000000,0x77e65c9bf35bfc436c818062fbc1ff5fe09fb11c,0.0,0,0,1681938781,DelayedOffchain,Pending,0x0000000000000000000000000000000000000000
2,D-FTM-0x1ab19e3e0bdbf77a8abfed1b2ede84507e88a1...,4.501193e+03,0x7346544d504552500000000000000000000000000000...,0x1ab19e3e0bdbf77a8abfed1b2ede84507e88a10f,0x0000000000000000000000000000000000000000,0x1ab19e3e0bdbf77a8abfed1b2ede84507e88a10f,0.0,0,0,1681938751,DelayedOffchain,Pending,0x0000000000000000000000000000000000000000
3,D-SOL-0xfc4a2ea9f786fce370ac9a9e5ba58008e7355d...,6.619156e+02,0x73534f4c504552500000000000000000000000000000...,0xfc4a2ea9f786fce370ac9a9e5ba58008e7355d43,0x0000000000000000000000000000000000000000,0xfc4a2ea9f786fce370ac9a9e5ba58008e7355d43,0.0,0,0,1681938721,DelayedOffchain,Filled,0x57cbb5aa0acd5b1bd823e1f25e6eb2354faaf532
4,D-sETH-0x2d476964a1895f748724ca9f38ec9eacf3ecc...,1.264603e+00,0x73455448504552500000000000000000000000000000...,0x2d476964a1895f748724ca9f38ec9eacf3eccb18,0x0000000000000000000000000000000000000000,0x2d476964a1895f748724ca9f38ec9eacf3eccb18,0.0,0,0,1681938706,DelayedOffchain,Filled,0xe164b79cade8714ad1545bcfea532236bc3bc2ad
5,D-sETH-0xfdda9e6647f18e75027be1f7f31ff03993d6c...,1.012200e+00,0x73455448504552500000000000000000000000000000...,0xfdda9e6647f18e75027be1f7f31ff03993d6c1ee,0x0000000000000000000000000000000000000000,0xfdda9e6647f18e75027be1f7f31ff03993d6c1ee,0.0,0,0,1681938706,DelayedOffchain,Filled,0xe164b79cade8714ad1545bcfea532236bc3bc2ad
6,D-LINK-0x1890a175f148affa5f36c3731de6507724a73...,3.000000e+03,0x734c494e4b5045525000000000000000000000000000...,0x1890a175f148affa5f36c3731de6507724a73792,0x0000000000000000000000000000000000000000,0x1890a175f148affa5f36c3731de6507724a73792,0.0,0,0,1681938691,DelayedOffchain,Filled,0xcdb2aea754eae02b8a556743afa0a03f16c24aa3
7,D-sBTC-0x1e67cc8857761f8a1fad023149330848be06e...,-4.953000e-01,0x73425443504552500000000000000000000000000000...,0x1e67cc8857761f8a1fad023149330848be06ea2c,0x0000000000000000000000000000000000000000,0x1e67cc8857761f8a1fad023149330848be06ea2c,0.0,0,0,1681938676,DelayedOffchain,Filled,0x57cbb5aa0acd5b1bd823e1f25e6eb2354faaf532
8,D-sETH-0x1ecb4bc56d52e8f6c22b674a78602a3ad413d...,-7.788400e+00,0x73455448504552500000000000000000000000000000...,0x1ecb4bc56d52e8f6c22b674a78602a3ad413d066,0x0000000000000000000000000000000000000000,0x1ecb4bc56d52e8f6c22b674a78602a3ad413d066,0.0,0,0,1681938676,DelayedOffchain,Filled,0x241754b86d6634fb6e7beb501845fa2361d61264
9,D-SOL-0x1ab19e3e0bdbf77a8abfed1b2ede84507e88a1...,2.352295e+02,0x73534f4c504552500000000000000000000000000000...,0x1ab19e3e0bdbf77a8abfed1b2ede84507e88a10f,0x0000000000000000000000000000000000000000,0x1ab19e3e0bdbf77a8abfed1b2ede84507e88a10f,0.0,0,0,1681938676,DelayedOffchain,Filled,0xe164b79cade8714ad1545bcfea532236bc3bc2ad
